# 🤖 Federated Learning: Basic Simulation
Simulates federated learning using a simple dataset.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

# Load data
df = pd.read_csv("../data/synthetic_data.csv")
X, y = df[['amount']].values.astype(np.float32), df['is_fraud'].values.astype(np.int32)
client_1 = (X[:3], y[:3])
client_2 = (X[3:], y[3:])

def create_tf_dataset(X, y):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(2)

train_data = [create_tf_dataset(*client_1), create_tf_dataset(*client_2)]

def model_fn():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return tff.learning.models.from_keras_model(
        model,
        input_spec=train_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

fed_avg = tff.learning.algorithms.build_weighted_fed_avg(model_fn)
state = fed_avg.initialize()
for round_num in range(1, 4):
    result = fed_avg.next(state, train_data)
    state = result.state
    print(f"Round {round_num}: {result.metrics}")